# Notebook- Model K15 - Custom Loss Function 
# Author : V.Albors   Date : 05.04.2020
# Purpose : Create Custom Loss Function 



**Input** :  
  * CSV files that identify the images to use as train and validation. CSV files are in directory csv_dir   
  * Images from train and validation. Images are in directory : imag_dir  
  * Saved model. Model is in directory : model_bin_dir  
  
**Output**:  
  * Download of the model trained with train dataset - 
  * Download the history of the model in order to be evaluated 

**Process**:  
 * Read Train and Validation images ( identified in the .csv files ) from the imag_dir directory   
 * Create a train and validation input & label tensors (no augmentation)
 * Define the architecture of model : 
                        
 * Train the model with the train dataset with callbacks (  ModuleCheckPoint , Early Stopping)
 * Save the trained model and history of the model in directory model_bin_dir 
 * Paint the Accuracy and Loss curves
 * Create results : Metrics 
 

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf

tf.keras.backend.clear_session()  # Reset

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.experimental.list_physical_devices('GPU') 
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
tf.config.experimental.set_memory_growth(physical_devices[0], True)
tf.debugging.set_log_device_placement(True)

Num GPUs Available:  1


In [3]:
#Tensorflow version 
print(tf.__version__)
from tensorflow.python.platform import build_info as tf_build_info
print(tf_build_info.cuda_version_number)
# Cuda Version 9.0 in v1.10.0
print(tf_build_info.cudnn_version_number)
# CudNN 7 in v1.10.0

2.0.0
10.0
7.6


In [4]:
# Define the name of the model 
Model_directory = "MODELK15"
Model_name = "ModelK15_0"
TRAIN = True

In [5]:
# Import routines
import sys  
subrc_dir = "/home/valborsf/Documents/UOC/PFMProject/"

sys.path.append(subrc_dir)  
from  Models_routines import *
import inspect

# List functions inside the module
import Models_routines as module
functions = inspect.getmembers(module, inspect.isfunction)
lsfunctions = [item[0] for item in functions]
print ( lsfunctions )

['confusion_ROC_AUC', 'create_column_tensor', 'create_label_tensor', 'create_val_test', 'define_dirs', 'extract_images_bm', 'extract_images_train', 'load_hist_model', 'load_images', 'load_images_tf', 'model_load', 'model_load_tf', 'plot_save_acc_loss', 'print_network', 'process_clinical_info', 'read_dataframes', 'read_dataframes_tables', 'reproducible_results', 'save_model', 'save_model_no_opt', 'save_network_json', 'start', 'stop', 'to_one_hot', 'to_one_hot_words', 'xi_squared']


In [6]:
# Reproducible results 
reproducible_results ()

In [7]:
# Define directories
(root_dir,json_dir,imag_dir,csv_dir,model_json_dir,model_bin_dir,results_dir,Tensor_dir) = define_dirs(Model_directory)

In [8]:
# New dataset without SONIC disturbing images
json_dir =  root_dir +"/DataNew/ALL_JSON/"                # .json dir images
imag_dir =  root_dir +"/DataNew/ALL_IMAGES/"              # .png dir - images

# directories for  CSV's
csv_dir =  root_dir +"/DataNew4/CSV/"                      # .csv dir - dftrain, dfval, dftest

In [9]:
# Load train,validation & Test 
(dftrain, dfval, dftest) = read_dataframes(csv_dir)

/home/valborsf/Documents/UOC/PFMProject/DataNew4/CSV/


In [10]:
# Define the label tensor
train_label_tensor = create_label_tensor(dftrain)
val_label_tensor = create_label_tensor(dfval)
#test_label_tensor = create_label_tensor(dftest)

In [11]:
# Create tensors from images
# Load Images 
height_imag = 150 
width_imag = 150
# This step is very time consuming !!!!!!!
train_image_tensor  = load_images_tf(dftrain,height_imag,width_imag)
val_image_tensor  = load_images_tf(dfval,height_imag,width_imag)
#test_image_tensor  = load_images(dftest,height_imag,width_imag)

Executing op ReadFile in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op DecodeJpeg in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Cast in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ExpandDims in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ResizeBilinear in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Squeeze in device /job:localhost/replica:0/task:0/device:CPU:0
(2100, 150, 150, 3)
(700, 150, 150, 3)


In [12]:
print ( train_image_tensor.shape)
print ( train_image_tensor.ndim)
slice_10_pictures = train_image_tensor[0:10]
slice_10_labels = train_label_tensor [0:10]
val_10_pictures = val_image_tensor[0:10]
val_10_labels = val_label_tensor [0:10]
print ( slice_10_pictures.shape)
print ( slice_10_pictures.ndim)
print ( slice_10_labels)
print ( slice_10_labels.shape)
print ( slice_10_labels.ndim)

(2100, 150, 150, 3)
4
(10, 150, 150, 3)
4
[0. 1. 0. 0. 1. 0. 0. 1. 1. 0.]
(10,)
1


In [13]:
#import keras
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import ModelCheckpoint


# Model 1 : Image to Predict Melanoma

image_inputs = Input(shape=(height_imag,width_imag,3))
m2 = layers.Conv2D(64, (3,3), activation='relu')(image_inputs)
m3 = layers.MaxPooling2D((2,2))(m2)
m4 = layers.Conv2D(32, (3,3),  activation='relu')(m3)
m5 = layers.MaxPooling2D((2,2))(m4)
m6 = layers.Conv2D(128, (3,3), activation='relu')(m5)
m7 = layers.MaxPooling2D((2,2))(m6)
m8 = layers.Flatten()(m7)
m9 = layers.Dense (512, activation='relu')(m8)
m10 = layers.Dropout(0.23)(m9)
benign_malign = layers.Dense (1, activation='sigmoid')(m10)


# Model instantiation
model = Model(image_inputs, benign_malign)



Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/tas

In [14]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 148, 148, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        18464     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       36992     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0     

In [15]:
#Print Network 
print_network (results_dir, model, Model_name)
#Save Network 
save_network_json (model_json_dir, model, Model_name)

In [16]:
# Define custom loss
tf.config.experimental_run_functions_eagerly(True)
from keras import backend as K
def custom_loss(layer):

    # Create a loss function that adds the MSE loss to the mean of all squared activations of a specific layer
    def loss(y_true,y_pred):
#        return K.mean(K.square(y_pred - y_true) + K.square(layer), axis=-1)
        print ( '\n Y true', y_true, '\n Y_pred', y_pred)
#        print ( '\n K mean:',K.mean(K.square(y_pred - y_true) , axis=-1))
        print ( '\n Loss Score :',K.mean(K.square(y_pred - y_true)))
#        return K.mean(K.square(y_pred - y_true) , axis=-1)
        return K.mean(K.square(y_pred - y_true))
      
   
    # Return a function
    return loss

Using TensorFlow backend.


In [ ]:
# Added to document !!!
# Define custom loss

from keras import backend as K
# Create a Custom loss function  MSE loss 
    def custom_loss(y_true,y_pred):
    # y_true = ground truth
    # y_pred = predicted y 
        return K.mean(K.square(y_pred - y_true) , axis=-1)       
    # Return a function
    return loss

# Model Compile
from tensorflow.keras import optimizers
model.compile ( loss='binary_crossentropy',
#               optimizer = optimizers.RMSprop(lr=1e-4),
               optimizer = optimizers.Adam(lr=1e-4),
               metrics= ['acc'])

In [17]:
# Model Compile
from tensorflow.keras import optimizers
model.compile ( loss=custom_loss(m3),
#               optimizer = optimizers.RMSprop(lr=1e-4),
               optimizer = optimizers.Adam(lr=1e-4),
               metrics= ['acc'])

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0


In [ ]:
## Added to document !!!!

# Model Compile
from tensorflow.keras import optimizers
model.compile ( loss=custom_loss,
               optimizer = optimizers.Adam(lr=1e-4),
               metrics= ['acc'])

In [18]:
# Callbacks
callbacks_list = [
# EarlyStopping when the model does not improve in loss

         tf.keras.callbacks.EarlyStopping (
             monitor = 'val_loss',             # Monitors the accuracy
             verbose=1,                        # log when finishes
             patience = 4,),                   # Interrupt if acc no improve in 4 epochs

#  ModelCheckpoint to store the weights of the best performing epoch. 
    
         tf.keras.callbacks.ModelCheckpoint(filepath=model_bin_dir+"Best_weights"+Model_name+".hdf5", 
             monitor = 'val_loss', # Won't overwritte the model file unless val_loss has
             verbose=1,            # improve 
             save_best_only=True),
         
#         keras.callbacks.TensorBoard(
#             log_dir =  Tensor_dir, 
#            histogram_freq = 1,
#            )
#        tf.keras.callbacks.ProgbarLogger(count_mode='samples')             # Display of log show samples seen 
                ]

In [19]:
# Model fit
epochs= 7
#batch_size = 64
batch_size = 2
import time
start_time = time.time()
history = model.fit (
#           train_image_tensor, 
#           train_label_tensor, 
           slice_10_pictures, 
           slice_10_labels, 
           epochs = epochs, 
           batch_size = batch_size, 
           callbacks=callbacks_list,
#           validation_data =( val_image_tensor,val_label_tensor))
           validation_data =( val_10_pictures ,val_10_labels))

elapsed_time = time.time() - start_time

print( time.strftime('Time spent in training :'"%H:%M:%S", time.gmtime(elapsed_time)))

save_model(model, history, model_bin_dir, Model_name)

Executing op RangeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op TensorDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ZipDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op

10/10 [==============================] - 2s 224ms/sample - loss: 0.3378 - acc: 0.4000 - val_loss: 0.2084 - val_acc: 0.7000
Epoch 2/7

 Y true tf.Tensor(
[[1.]
 [0.]], shape=(2, 1), dtype=float32) 
 Y_pred tf.Tensor(
[[0.36076725]
 [0.32756644]], shape=(2, 1), dtype=float32)

 Loss Score : tf.Tensor(0.25795913, shape=(), dtype=float32)
 2/10 [=====>........................] - ETA: 0s - loss: 0.2580 - acc: 0.5000
 Y true tf.Tensor(
[[0.]
 [1.]], shape=(2, 1), dtype=float32) 
 Y_pred tf.Tensor(
[[0.37425792]
 [0.37691358]], shape=(2, 1), dtype=float32)

 Loss Score : tf.Tensor(0.26415285, shape=(), dtype=float32)

 Y true tf.Tensor(
[[0.]
 [1.]], shape=(2, 1), dtype=float32) 
 Y_pred tf.Tensor(
[[0.42936048]
 [0.41905406]], shape=(2, 1), dtype=float32)

 Loss Score : tf.Tensor(0.2609243, shape=(), dtype=float32)

 Y true tf.Tensor(
[[0.]
 [0.]], shape=(2, 1), dtype=float32) 
 Y_pred tf.Tensor(
[[0.45740288]
 [0.48509234]], shape=(2, 1), dtype=float32)

 Loss Score : tf.Tensor(0.22226599, 

 8/10 [=======================>......] - ETA: 0s - loss: 0.2569 - acc: 0.6250
 Y true tf.Tensor(
[[0.]
 [0.]], shape=(2, 1), dtype=float32) 
 Y_pred tf.Tensor(
[[0.370819 ]
 [0.4306744]], shape=(2, 1), dtype=float32)

 Loss Score : tf.Tensor(0.1614936, shape=(), dtype=float32)

 Y true tf.Tensor(
[[1.]
 [0.]], shape=(2, 1), dtype=float32) 
 Y_pred tf.Tensor(
[[0.4110507 ]
 [0.37938216]], shape=(2, 1), dtype=float32)

 Loss Score : tf.Tensor(0.24539606, shape=(), dtype=float32)

 Y true tf.Tensor(
[[0.]
 [0.]], shape=(2, 1), dtype=float32) 
 Y_pred tf.Tensor(
[[0.37834734]
 [0.40938532]], shape=(2, 1), dtype=float32)

 Loss Score : tf.Tensor(0.15537152, shape=(), dtype=float32)

 Y true tf.Tensor(
[[0.]
 [0.]], shape=(2, 1), dtype=float32) 
 Y_pred tf.Tensor(
[[0.39660203]
 [0.42293903]], shape=(2, 1), dtype=float32)

 Loss Score : tf.Tensor(0.1680853, shape=(), dtype=float32)

 Y true tf.Tensor(
[[0.]
 [0.]], shape=(2, 1), dtype=float32) 
 Y_pred tf.Tensor(
[[0.39249283]
 [0.3798088 ]]

In [ ]:
# Import Model Test if not need to Train 
if not TRAIN :
    model = model_load ( model_bin_dir, Model_name)

In [ ]:
# Display curves of loss and accuracy during training and save results 
plot_save_acc_loss(results_dir, history.history, Model_name)

In [ ]:
# Load Model 
# Load weights 
# Import with this compatibility to solve the problem 
# module 'tensorflow' has no attribute 'placeholder'`.

 
#model = build_model()
#model.load_weights('my_weights.model')
TRAIN = False
if not TRAIN :
    Model_name = "ModelK15_0"
    model = model_load_tf ( model_bin_dir, Model_name)
    model.load_weights(model_bin_dir+"Best_weights"+Model_name+".hdf5")

In [ ]:
# Create tensors from images
# Load Images 

# This step is very time consuming !!!!!!!

test_image_tensor  = load_images_tf(dftest,height_imag,width_imag)

In [ ]:
# Define the label tensor
test_label_tensor = create_label_tensor(dftest)

In [ ]:
#  ROC /AUC
scores = model.evaluate(test_image_tensor, test_label_tensor, verbose=2)

In [ ]:
model.metrics_names
scores

In [ ]:
# Loss 0.50  Acc=0.76

In [ ]:
# Compute predictions
# This takes time !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
y_pred_keras = model.predict(test_image_tensor).ravel()   # y_pred_probabilities
y_pred = y_pred_keras > 0.5                                # y_pred_class : if >0.5  = True => Malignant
y_test = dftest.bm.astype('category').cat.codes            # Ground truth 
class_labels= dftest.bm.unique()                           # Labels of ground truth 


In [ ]:
#Print Metrics + Confusion ROC AUC
confusion_ROC_AUC ( y_test, y_pred, y_pred_keras, class_labels, results_dir, Model_name )

In [ ]:
tf.keras.backend.clear_session()  # Reset